# Handwriting Recognition

## Cell below to be deleted after running once

In [9]:
pip install numpy pandas keras opencv-python imutils matplotlib seaborn scikit-learn


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB 196.9 kB/s eta 0:03:14
   ---------------------------------------- 0.1/38.1 MB 302.7 kB/s eta 0:02:06
   ---------------------------------------- 0.1/38.1 MB 327.7 kB/s eta 0:01:56
   ---------------------------------------- 0.1/38.1 MB 379.3 kB/s eta 0:01:41
   ---------------------------------------- 0.2/38.1 MB 513.4 kB/s eta 0:01:14
   ------------------------------

## Importing necessary packages

In [7]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random 
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

## Preprocessing the data

### Build the dataframe of image of 64x255 and corresponding label

In [34]:
def resize_image(img):
    (h, w) = img.shape
    final_img = np.ones([64, 256]) * 255  #blank white image

    # crop
    if w > 256:
        img = img[:, :256]
        
    if h > 64:
        img = img[:64, :]

    final_img[:h, :w] = img
    return final_img


def get_dataframe(directory, csv_filepath):
    
    #Loading the csv files containing the labels
    labels_df = pd.read_csv(csv_filepath)
    img_size = 32
    data = []
    
    count = 0 #counter of the number of files to put into the dataframe
    
    for img_filename in os.listdir(directory):
        if count >= 500:
            break
        
        #Get image in grayscale
        img_path = os.path.join(directory, img_filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        #Resize the image
        img = resize_image(img)
        img = img / 255 #normalization
        
        #Find corresponding label for the image in the csv file
        label = labels_df.loc[labels_df["FILENAME"] == img_filename, "IDENTITY"].values
        
        #Check if the label is not 'UNREADABLE'
        if label[0] != 'UNREADABLE':
            #Add the image and its label to the data
            data.append([img, label[0]])
            count += 1
    
    #Build the dataframe
    df = pd.DataFrame(data, columns=["Image", "Label"])
    print(df.head())
    
    return df

### Visualize the data

In [35]:
def visualize_the_first_image(df):
    plt.imshow(df["Image"][0], cmap='gray')
    plt.title(f"Label: {df['Label'][0]}")
    plt.show()

In [ ]:
train_df = get_dataframe('dataset/train', 'dataset/written_name_train_v2.csv')

In [ ]:
visualize_the_first_image(train_df)